## 11단계: 가변 길이 인수(순전파 편)

> 지금까지 우리는 함수에 입출력 변수가 하나씩인 경우만 생각했지만, 입출력이 여러 개인 경우도 존재합니다. \
이 점을 고려하여 DeZero가 가변 길이 입출력을 처리할 수 있도록 확장하려 합니다.

<img src="images/그림 11-1.png" width=500/>
<br/>
<img src="images/그림 11-2.png" width=500/>

### 11.1 Function 클래스 수정

`__call__` 메서드의 인수와 반환값을 리스트로 바꿔보자.

In [7]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x

In [8]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()

### 11.2 Add 클래스 구현

새로운 Function 클래스를 상속하여 Add 클래스를 구현해보자. \
이 때, 인수와 반환값이 `iterable` 객체여야 한다.

In [9]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

xs = [Variable(np.array(2)), Variable(np.array(3))]  # 리스트로 준비
f = Add()
ys = f(xs)  # ys 튜플
y = ys[0]
print(y.data)

5


이제 순전파에 한해 가변 길이 인수와 반환값에 대응할 수 있게 되었지만, 다소 귀찮은 부분이 있다. \
입력 변수를 반드시 리스트에 담아 건네줘야 한다고 요구해야 하고, 반환값으로 튜플을 받게 하는 것도 자연스럽지 않다. \
12단계에서는 이 부분을 개선해본다.